# Importing Libraries

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import numpy
import pandas as pd
import os
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix
import torch.distributed as dist
from torch.utils.data import DataLoader
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from torch.utils.data import DataLoader, TensorDataset
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import keras
import torch
import transformers
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout

# Importing the dataset and change the Label to integer format

In [ ]:
data = pd.read_csv('/content/train (2).csv',header = None,delimiter=',')
data.columns = ['id','Text', 'Label']
data.drop(0,axis=0,inplace=True)
data['Label'].replace({'NOT': 0, 'OFF': 1}, inplace=True)
data2=pd.read_csv('/content/test (2).csv',header = None,delimiter=',')
data2.columns = ['id','Text', 'Label']
data2.drop(0,axis=0,inplace=True)
data2['Label'].replace({'NOT': 0, 'OFF': 1}, inplace=True)
data.shape
data.head()

,id,Text,Label
1,42884,@USER I’m done with you as well. An INTENTIONA...,0
2,92152,I now have over 6k followers. Only 94k to go ...,0
3,65475,@USER Tom was bought! He is more interested in...,0
4,22144,@USER @USER Even her brother thinks she is a m...,1
5,81048,@USER @USER @USER @USER @USER I can understand...,1


,id,Text,Label
1,42884,@USER I’m done with you as well. An INTENTIONA...,0
2,92152,I now have over 6k followers. Only 94k to go ...,0
3,65475,@USER Tom was bought! He is more interested in...,0
4,22144,@USER @USER Even her brother thinks she is a m...,1
5,81048,@USER @USER @USER @USER @USER I can understand...,1


In [ ]:
print("___________________________________________Train_________________________________________________________")
print(data.Label.value_counts())
print("________________________________Test________________________________________________________________________")
print(data2.Label.value_counts())
print("____________________________________________________________________________________________________")
nltk.download('punkt')
nltk.download('stopwords')

___________________________________________Train_________________________________________________________
0    8221
1    4092
Name: Label, dtype: int64
________________________________Test________________________________________________________________________
0    620
1    240
Name: Label, dtype: int64
____________________________________________________________________________________________________


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Preprocess the data

In [ ]:
# Preprocess the text by removing special characters, converting to lowercase, etc.
def preprocess_text(text):
    # Remove non-alphanumeric characters and convert to lowercase
    text = re.sub('[^0-9a-zA-Z]+', ' ', text).lower()
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Remove stop words
    tokens = [token for token in tokens if token not in nltk.corpus.stopwords.words('english')]
    # Join the tokens back into a string
    return ' '.join(tokens)

data['Text'] = data['Text'].fillna('')
data['Text'] = data['Text'].apply(preprocess_text)

data2['Text'] = data2['Text'].fillna('')
data2['Text'] = data2['Text'].apply(preprocess_text)

In [ ]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(data['Text'])
sequences_train = tokenizer.texts_to_sequences(data['Text'])
tokenizer.fit_on_texts(data2['Text'])
sequences_test = tokenizer.texts_to_sequences(data2['Text'])

In [ ]:
padded_sequences_train = pad_sequences(sequences_train, maxlen=100)
padded_sequences_test = pad_sequences(sequences_test, maxlen=100)


# Designing the CNN Model

In [ ]:
# Build the model
model2 = Sequential()
model2.add(Embedding(10000, 128, input_length=100))
model2.add(Conv1D(64, 5, activation='relu'))
model2.add(GlobalMaxPooling1D())
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 128)          1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 96, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                      

In [ ]:
model2.fit(padded_sequences_train, data['Label'], epochs=10, batch_size=32, validation_data=(padded_sequences_test, data2['Label']))
loss, accuracy = model2.evaluate(padded_sequences_test, data2['Label'])
print('Test loss:', loss)
print('Test accuracy:', accuracy)
y_pred = model2.predict(padded_sequences_test)
y_pred = numpy.argmax(y_pred, axis=1)
print(classification_report(data2['Label'], y_pred))
print(confusion_matrix(data2['Label'], y_pred))

Epoch 1/10
385/385 [==============================] - 38s 93ms/step - loss: 0.5692 - accuracy: 0.7223 - val_loss: 0.7147 - val_accuracy: 0.6453
Epoch 2/10
385/385 [==============================] - 28s 72ms/step - loss: 0.3844 - accuracy: 0.8420 - val_loss: 0.8133 - val_accuracy: 0.6326
Epoch 3/10
385/385 [==============================] - 18s 48ms/step - loss: 0.1794 - accuracy: 0.9394 - val_loss: 1.0916 - val_accuracy: 0.6477
Epoch 4/10
385/385 [==============================] - 18s 48ms/step - loss: 0.0812 - accuracy: 0.9748 - val_loss: 1.4286 - val_accuracy: 0.6453
Epoch 5/10
385/385 [==============================] - 21s 56ms/step - loss: 0.0484 - accuracy: 0.9870 - val_loss: 1.7401 - val_accuracy: 0.5733
Epoch 6/10
385/385 [==============================] - 19s 48ms/step - loss: 0.0385 - accuracy: 0.9892 - val_loss: 1.7128 - val_accuracy: 0.6081
Epoch 7/10
385/385 [==============================] - 19s 48ms/step - loss: 0.0307 - accuracy: 0.9911 - val_loss: 1.6207 - val_accuracy:

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Making BERT Model

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
# Tokenize the text and convert it to input features
train_data, test_data, train_labels, test_labels = train_test_split(data['Text'], data['Label'], test_size=0.2)
train_input = tokenizer(list(train_data), padding=True, truncation=True, max_length=128, return_tensors='pt')
test_input = tokenizer(list(test_data), padding=True, truncation=True, max_length=128, return_tensors='pt')
train_labels = torch.tensor(train_labels.tolist())
test_labels = torch.tensor(test_labels.tolist())

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_input['input_ids'])*2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
batch_size = 32
model.to(device)
train_dataset = TensorDataset(torch.tensor(train_input['input_ids']), torch.tensor(train_input['attention_mask']), torch.tensor(train_labels))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
optimizer.zero_grad()
scheduler.step()

<ipython-input-14-f494d63d3448>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(train_input['input_ids']), torch.tensor(train_input['attention_mask']), torch.tensor(train_labels))
/usr/local/lib/python3.9/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


In [ ]:
for epoch in range(5):
    print("{}/5 epoch_______________________".format(epoch+1))
    for batch in train_dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    scheduler.step()

1/5 epoch_______________________


# Evaluating the bert Model

In [ ]:
model.eval()
eval_input = test_input
eval_labels = test_labels
eval_dataset = TensorDataset(torch.tensor(eval_input['input_ids']), torch.tensor(eval_input['attention_mask']), torch.tensor(eval_labels))
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size)

In [ ]:
total_eval_loss = 0.0
total_eval_accuracy = 0.0
def flat_accuracy(preds, labels):
    pred_flat = numpy.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return numpy.sum(pred_flat == labels_flat) / len(labels_flat)

for batch in eval_dataloader:
    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    labels = batch[2].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

    total_eval_loss += loss.item()
    preds = logits.detach().cpu().numpy()
    label_ids = labels.to('cpu').numpy()
    total_eval_accuracy += flat_accuracy(preds, label_ids)

In [ ]:
avg_eval_loss = total_eval_loss / len(eval_dataloader)
avg_eval_accuracy = total_eval_accuracy / len(eval_dataloader)

print("Evaluation loss: {0:.4f}".format(avg_eval_loss))
print("Evaluation accuracy: {0:.4f}".format(avg_eval_accuracy))